In [54]:
queryVar = ['Fraud']
evidenceVar = []
hiddenVar = ['Trav']


import pandas as pd

class factor:
    def __init__(self, path=None, table=None):
        if path:
            self.table = pd.read_csv(path)
        else:
            self.table=table
       
    def __str__(self):
        return self.table.to_string()
    
    def sumOfTable(self):
        return sum(self.table['value'])
    
        
def productFactor(fact1, fact2):
    fact1 = fact1.table.rename(columns={'value': 'factor_left'}, inplace=False)
    fact2 = fact2.table.rename(columns={'value': 'factor_right'}, inplace=False)
    new_factor = fact1.merge(fact2)
    new_values = []
    
    for index, row in new_factor.iterrows():
        new_values.append(row['factor_left']*row['factor_right'])

    new_factor['value'] = new_values
    new_factor=new_factor.drop(columns=['factor_left','factor_right'])
    
    print(new_factor)
    return factor(table=new_factor)

    

In [57]:
def containsVariable(var,factor):
    return var in factor.table.columns
    
def printFactorList(l):
    for i in l:
        print(i.table.to_string())
        print("\n")
    print('Done')
    
def observeFactor(factor, variable, value):
    
    tableFilter = factor.table[variable] == value
    factor.table = factor.table[tableFilter]    
    return factor

def normalizedFactor(factor):
    columnSum = factor.table['value'].sum()
    factor.table = factor.table['value'].apply(lambda x: x/columnSum)
    return factor

def findAndObserveFactor(factorList,variable,value):
    factorToObserve = 0
    for factor in factorList :
        if(factor.table.columns[0] == variable):
            factorToObserve = factor
            break
    
    observeFactor(factorToObserve, variable, value)
    
def sumOut(factor, variable):
    table = factor.table
    new_factor_table = table.drop(columns=['value',variable], inplace=False).drop_duplicates()

    
    new_values = []
    
    for index, r in new_factor_table.iterrows():
        row=new_factor_table.filter(items = [index], axis=0)
        sum_rows = table.merge(row)
        new_values.append(sum_rows['value'].sum())

    
    new_factor_table['value'] = new_values
    factor.table = new_factor_table
    return factor
    
def joins(factorList,orderedListOfHiddenVariables, joinOrder):
    for var in joinOrder:
        
        #partition our factors
        joinList = [x for x in factorList if containsVariable(var,x)]
        factorList = [x for x in factorList if not containsVariable(var,x)]
        
        currFactor = joinList.pop(0)
        
        while(len(joinList) > 0):
            currFactor = productFactor(currFactor, joinList.pop(0))
        
        if var in orderedListOfHiddenVariables:
            currFactor = sumOut(currFactor,var)
            
        factorList.append(currFactor)
        
    return currFactor

def getVariables():
    
    new_factor_list = []
    
    file = open('config.txt', 'r')
    factors = file.readlines()[0].split(',')
    for f in factors:
        new_factor_list.append(factor(path= f+".txt"))
    return (new_factor_list, factors)
    
factorList, variables = getVariables()


def inQuery(var,queryVar, evidenceVar, hiddenVar):
    
    if var in queryVar:
        return True
    elif var in evidenceVar:
        return True
    elif var in hiddenVar:
        return True
    return False

def inference(factorList, queryVar, evidenceVar, hiddenVar):
    joinOrder = [x for x in variables if inQuery(x,queryVar, evidenceVar, hiddenVar)]
    print(joinOrder)    
    for evidence in evidenceVar:
        findAndObserveFactor(factorList, evidence[0],evidence[1])
    result = joins(factorList, hiddenVar, joinOrder)
    
    result = normalizedFactor(result)
    
    return result
    

resultFactor = inference(factorList, queryVar, evidenceVar, hiddenVar)
print(resultFactor.table)



['Trav', 'Fraud']
0     0.000008
1     0.000035
2     0.000407
3     0.001700
4     0.000005
5     0.000019
6     0.000410
7     0.001716
8     0.000270
9     0.004708
10    0.026736
11    0.466136
12    0.000027
13    0.000471
14    0.026979
15    0.470373
Name: value, dtype: float64


In [53]:
print(resultFactor.__str__())

0     0.000008
1     0.000035
2     0.000407
3     0.001700
4     0.000005
5     0.000019
6     0.000410
7     0.001716
8     0.000270
9     0.004708
10    0.026736
11    0.466136
12    0.000027
13    0.000471
14    0.026979
15    0.470373
